In [9]:
#!{sys.executable} -m pip install tpot

In [10]:
from tpot import TPOTClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

In [11]:
digits = load_digits()
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target,
                                                    train_size=0.75, test_size=0.25, random_state=42)


In [12]:
n_gen =2
pop_size=2
tpot = TPOTClassifier(generations=n_gen, population_size=pop_size, verbosity=2, random_state=42)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('tpot_digits_pipeline.py')

Optimization Progress:   0%|          | 0/6 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9428390472256643

Generation 2 - Current best internal CV score: 0.9495249896736887

Best pipeline: RandomForestClassifier(RandomForestClassifier(input_matrix, bootstrap=True, criterion=gini, max_features=0.15000000000000002, min_samples_leaf=7, min_samples_split=9, n_estimators=100), bootstrap=True, criterion=gini, max_features=0.2, min_samples_leaf=8, min_samples_split=4, n_estimators=100)
0.9688888888888889


/home/hadoop/.local/lib/python3.11/site-packages/sklearn/metrics/_scorer.py:765: FutureWarning: sklearn.metrics.SCORERS is deprecated and will be removed in v1.3. Please use sklearn.metrics.get_scorer_names to get a list of available scorers and sklearn.metrics.get_metric to get scorer.
  warnings.warn(


In [13]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import PolynomialFeatures
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive


In [27]:

# NOTE: Make sure that the outcome column is labeled 'target' in the data file
#tpot_data = pd.read_csv('./Data/titanic.csv', sep=',', dtype=np.float64)
tpot_data = pd.read_csv('./Data/titanic.csv', sep=',')

tpot_data = tpot_data.select_dtypes(exclude= 'object')
tpot_data.dropna(inplace=True)

In [28]:
tpot_data.head()

,pclass,survived,age,sibsp,parch,fare,body
3,1,0,30.0,1,2,151.5500,135.0
9,1,0,71.0,0,0,49.5042,22.0
10,1,0,47.0,1,0,227.5250,124.0
25,1,0,25.0,0,0,26.0000,148.0
39,1,0,48.0,0,0,50.4958,208.0


In [29]:
target_col = 'survived' 
features = tpot_data.drop(target_col, axis=1)
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data[target_col], random_state=42)


In [33]:
penalty = 'none'
# Average CV score on the training set was: 0.9799428471757372
exported_pipeline = make_pipeline(
    PolynomialFeatures(degree=2, include_bias=False, interaction_only=False),
    StackingEstimator(estimator=LogisticRegression(C=0.1, dual=False, penalty=penalty)),
    RandomForestClassifier(bootstrap=True, criterion="entropy", max_features=0.35000000000000003, min_samples_leaf=20, min_samples_split=19, n_estimators=100)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

/home/hadoop/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0